In [1]:
import time
import sys
import os
path_project = "\\".join(os.getcwd().split("\\")[:-1])
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, path_project)

from calibration.rfphasetune import RFPhaseTune


In [2]:
from hardware import config as hcf
from hardware.hardwaremanager import HardwareManager

hm = HardwareManager()
# adds synthesizer, laser, and pulse generator 
hm.add_default_hardware()


Connect to Pulse Streamer via JSON-RPC.
IP / Hostname: 192.168.1.26
Pulse Streamer 8/2 firmware: v1.7.2
Client software: v1.7.0


In [3]:
# Initialize experiment
rfphtuner = RFPhaseTune()

# Optional: Adjust paraset
rfphtuner.set_paraset(
    freq_mhz=600.0,
    power_dbm=0.0,
    tol_end=1e-3,
    phaseA_deg=0.0,
    initial_step_deg=10.0,
    min_step_deg=0.08,
    max_step_deg=20.0,
    rate_refresh=10.0
)
stop_time = 20
rfphtuner.set_stoptime(stop_time)
# rfphtuner.set_runnum(50)

# # Setup experiment
rfphtuner.start()
time.sleep(stop_time)

Skipping unpicklable attribute: _thread


In [4]:
import numpy as np
import plotly.graph_objects as go
from dash_bootstrap_templates import load_figure_template
PLOT_THEME = "sketchy"
load_figure_template([PLOT_THEME, PLOT_THEME + "_dark"])
# Example: decide the theme
# Replace this with your Dash input, e.g. switch_on
light_mode = False  # or False
# PLOT_THEME = "plotly"  # base theme name (can be "ggplot2", "simple_white", etc.)
template = PLOT_THEME if light_mode else PLOT_THEME + "_dark"

fig = go.Figure()
phases = rfphtuner.dataset["phase_history"]
powers = rfphtuner.dataset["power_history"]
iterations = np.arange(len(phases))
# Phase trace (left y-axis)
fig.add_trace(go.Scattergl(
    x=iterations,
    y=phases,
    name="Phase (deg)",
    mode="lines+markers",
    marker=dict(symbol="circle", size=10),
    yaxis="y1"
))

# Power trace (right y-axis)
fig.add_trace(go.Scattergl(
    x=iterations,
    y=powers,
    name="Power (uW)",
    mode="lines+markers",
    marker=dict(symbol="diamond", size=10),
    yaxis="y2"
))

# Layout with theme and clean margin/legend
fig.update_layout(
    xaxis=dict(title="Iteration"),
    yaxis=dict(title="Phase (deg)"),
    yaxis2=dict(
        title="Power (uW)",
        overlaying="y",
        side="right",
        showgrid=False,
    ),
    template=template,
    font=dict(size=21),
    legend=dict(
        bgcolor="rgba(0,0,0,0)",
        x=0.75,
        y=0.95,
    ),
    title=None,
    margin=dict(t=0)
    , width=750
)

fig.show()
